# Installing Dependencies

In [ ]:
!huggingface-cli login # LOGIN FIRST


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
! pip install -U -q accelerate transformers einops datasets peft bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

# Importing Dependencies

In [ ]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import os

# Finetuning

In [ ]:
# Load the model
model_name = "microsoft/phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1_5",
    device_map={"":0},
    trust_remote_code=True,
    quantization_config=bnb_config
)

print(model)

import bitsandbytes as bnb

def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split(".")
            # model-specific
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if "lm_head" in lora_module_names:  # needed for 16-bit
        lora_module_names.remove("lm_head")
    return list(lora_module_names)

print(find_all_linear_names(model))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (dense): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear4bit(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (final_layernor

In [ ]:

'''# Lora Config 1
config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)'''

# Lora Config 2
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()
print(model)

trainable params: 3,145,728 || all params: 1,421,416,448 || trainable%: 0.2213093850451912
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2048)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-23): 24 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
             

In [ ]:
import torch
from transformers import Conv1D

def get_specific_layer_names(model):
    # Create a list to store the layer names
    layer_names = []

    # Recursively visit all modules and submodules
    for name, module in model.named_modules():
        # Check if the module is an instance of the specified layers
        if isinstance(module, (torch.nn.Linear, torch.nn.Embedding, torch.nn.Conv2d, Conv1D)):
            # model name parsing

            layer_names.append('.'.join(name.split('.')[4:]).split('.')[0])

    return layer_names

list(set(get_specific_layer_names(model)))

['',
 '20',
 '0',
 '7',
 '21',
 '19',
 '13',
 '16',
 '23',
 '9',
 '18',
 '3',
 '4',
 '5',
 '17',
 '6',
 '8',
 '15',
 '22',
 '11',
 '12',
 '2',
 '10',
 '1',
 '14']

In [ ]:
from datasets import load_dataset, concatenate_datasets, Dataset
import torch
import random
import sympy as sp
import sqlite3
import pandas as pd
import sqlite3
import re
import numpy as np

In [ ]:
def tokenize(sample):
    model_inps =  tokenizer(sample["text"], padding=True, truncation=True, max_length=512)
    return model_inps

def format_options(options):
    formatted_options = ', '.join(options)
    return formatted_options

def load_process_tokenize(num_rows, data_df, teacher=False):
    data_df = data_df.iloc[:num_rows].copy()

    if teacher:
        data_df["text"] = data_df.apply(lambda x: "Instruction: " + x["question"] + " options: " + format_options(x["options"]) + "\Output: " + x["LLMs_rationale"], axis=1)
    else:
        data_df["text"] = data_df.apply(lambda x: "Instruction: " + x["question"] + " options: " + format_options(x["options"]) + "\Output: " + x["rationale"], axis=1)

    processed_data = Dataset.from_pandas(data_df)
    tokenizer.pad_token = tokenizer.eos_token
    tokenized_data = processed_data.map(tokenize, batched=True, desc="Tokenizing data", remove_columns=processed_data.column_names)

    return tokenized_data

def load_db(database_name, table_name, num_rows, seed=42):

    conn = sqlite3.connect(database_name)
    query = f"SELECT * FROM {table_name}"
    df_custom = pd.read_sql_query(query, conn)
    conn.close()
    df_custom_shuffled = df_custom.sample(frac=1, random_state=seed).reset_index(drop=True)
    df_custom_reduced = df_custom_shuffled.head(num_rows)

    return df_custom_reduced

In [ ]:
data_df.head()

,question,options,rationale,correct
0,"Two friends plan to walk along a 43-km trail, ...","[A)21, B)21.5, C)22, D)22.5, E)23]","If Q complete x kilometers, then P completes 1...",E
1,"In the coordinate plane, points (x, 1) and (5,...","[A)4 and 1, B)1 and 5, C)5 and 1, D)3 and 5, E...",Line k passes through the origin and has slope...,C
2,"For all numbers p and q, the operation @ is de...","[A)II, B)I and II, C)I and III, D)II and III, ...",p@q = p^2 - pq=p(p-q).... so p@q will be zero ...,B
3,Carl is facing very difficult financial times ...,"[A)$1600, B)$2000, C)$2150, D)$2500, E)$12000]","Usually, you are given the annual rate of inte...",A
4,The speed at which a man can row a boat in sti...,"[A)18 seconds, B)27 seconds, C)26 seconds, D)1...",Speed of the boat downstream = 25 +11\n= 36 km...,E


In [ ]:
# AQUA RAT TRAINING DATASET
data = load_dataset("aqua_rat", split="train")
data_df = data.to_pandas()
tokenized_data = load_process_tokenize(500, data_df)

# do all with 2 epochs

Generating train split:   0%|          | 0/97467 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/254 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/254 [00:00<?, ? examples/s]

Tokenizing data:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# TEACHER LLM TRAINING DATASET - UPLOAD TEACHER DATASET TO COLAB FIRST
n_teacher = 500
teacher_df = load_db("/content/teacher_llm_dataset.db", "LLM_results", n_teacher, seed=42)
tokenized_data = load_process_tokenize(n_teacher, teacher_df, teacher=True)

Tokenizing data:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# SET MODEL NAME HERE
finetuned_model_name = 'phi-1_5-finetuned-aqua-rat-0.5k'

training_arguments = TrainingArguments(
        output_dir=finetuned_model_name,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        #max_steps=1000,
        num_train_epochs=2, # ORIGINALY 1
        push_to_hub=True
    )

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()
trainer.push_to_hub()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
100,1.457000
200,1.374100


CommitInfo(commit_url='https://huggingface.co/rk68/phi-1_5-finetuned-aqua-rat-0.5k/commit/a5ab4f7e7c7b30f3cc345d524ba43ab4bb729be8', commit_message='End of training', commit_description='', oid='a5ab4f7e7c7b30f3cc345d524ba43ab4bb729be8', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from IPython.display import HTML, display

def disconnect_runtime():
  display(HTML('<script>google.colab.kernel.disconnect()</script>'))

disconnect_runtime()

In [ ]:
# TEACHER LLM TRAINING DATASET - UPLOAD TEACHER DATASET TO COLAB FIRST
n_teacher = 2000
teacher_df = load_db("/content/teacher_llm_dataset.db", "LLM_results", n_teacher, seed=42)
tokenized_data = load_process_tokenize(n_teacher, teacher_df, teacher=True)

# SET MODEL NAME HERE
finetuned_model_name = 'phi-1_5-finetuned-aqua-rat-teacher-AM-2k'

training_arguments = TrainingArguments(
        output_dir=finetuned_model_name,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        #max_steps=1000,
        num_train_epochs=2, # ORIGINALY 1
        push_to_hub=True
    )

trainer = Trainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()
trainer.push_to_hub()

Tokenizing data:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
100,0.750400
200,0.734700
300,0.742800
400,0.734500
500,0.732800
600,0.680400
700,0.682300
800,0.701000
900,0.680900
1000,0.683200


CommitInfo(commit_url='https://huggingface.co/rk68/phi-1_5-finetuned-aqua-rat-teacher-AM-2k/commit/bb1562e23d83591f24521017cc1a8b802e962c6f', commit_message='End of training', commit_description='', oid='bb1562e23d83591f24521017cc1a8b802e962c6f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_name = "microsoft/phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


# List of r values to iterate over
r_values = [24,32,40]

#data = load_dataset("aqua_rat", split="train")
#data_df = data.to_pandas()

n_teacher = 2000
teacher_df = load_db("/content/teacher_llm_dataset.db", "LLM_results", n_teacher, seed=42)
tokenized_data = load_process_tokenize(n_teacher, teacher_df, teacher=True)

for r in r_values:
    # Quantization config
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )

    # Load the base model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map={"": 0},
        trust_remote_code=True,
        quantization_config=bnb_config
    )

    # Lora Config
    config = LoraConfig(
        r=r,
        lora_alpha=16,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    # Get PEFT model with current Lora config
    model = get_peft_model(model, config)

    # Optional: Print trainable parameters or any other details
    model.print_trainable_parameters()


    # Define training arguments
    finetuned_model_name = f'phi-1_5-finetuned-aqua-rat-AM-teacher-2k-r-{r}'
    training_arguments = TrainingArguments(
        output_dir=finetuned_model_name,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        num_train_epochs=2,
        push_to_hub=True
    )

    trainer = Trainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
    )
    trainer.train()
    trainer.push_to_hub()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Tokenizing data:   0%|          | 0/2000 [00:00<?, ? examples/s]

trainable params: 4,718,592 || all params: 1,422,989,312 || trainable%: 0.3315971497613047


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
100,1.022900
200,0.798400
300,0.785600
400,0.767200
500,0.762400
600,0.710300
700,0.710700
800,0.728600
900,0.707900
1000,0.711100


...

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
100,1.024400
200,0.799000
300,0.786500
400,0.767600
500,0.762500
600,0.710600
700,0.710800
800,0.728300
900,0.707800
1000,0.711400


trainable params: 7,864,320 || all params: 1,426,135,040 || trainable%: 0.5514428703750243


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
100,1.024300
200,0.798200
300,0.785900
400,0.767200
500,0.762400
600,0.710300
700,0.710800
800,0.728200
900,0.707800
1000,0.711200


In [ ]:
# Model names for different sample sizes
finetuned_model_names = {
    1000: 'phi-1_5-finetuned-aqua-rat-AM-1k',
    3000: 'phi-1_5-finetuned-aqua-rat-AM-3k',
    4000: 'phi-1_5-finetuned-aqua-rat-AM-4k',
    5000: 'phi-1_5-finetuned-aqua-rat-AM-5k',
}

# Loop through the sample sizes and their corresponding model names
for sample_size, model_name in finetuned_model_names.items():
    # Load and process the dataset
    data = load_dataset("aqua_rat", split="train")
    data_df = data.to_pandas()
    tokenized_data = load_process_tokenize(sample_size, data_df) # Adjust the sample size here

    # Update the training arguments with the current model name
    training_arguments = TrainingArguments(
        output_dir=model_name,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        num_train_epochs=2, # Adjusted to 2 as mentioned
        push_to_hub=True
    )

    # Assuming model and tokenizer are defined elsewhere
    trainer = Trainer(
        model=model,
        train_dataset=tokenized_data,
        args=training_arguments,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
    )

    # Train and push to hub
    trainer.train()
    trainer.push_to_hub()


Tokenizing data:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
100,1.828600
200,1.793000
300,1.738800
400,1.673600
500,1.699700


Tokenizing data:   0%|          | 0/3000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
100,1.580500
200,1.581700
300,1.619600
400,1.634800
500,1.637800
600,1.611900
700,1.589500
800,1.600200
900,1.617900
1000,1.589600


Tokenizing data:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
100,1.505400
200,1.543800
300,1.548000
400,1.567200
500,1.564700
600,1.544900
700,1.567300
800,1.548800
900,1.556100
1000,1.511800


In [ ]:
model_name = "microsoft/phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# List of lora_alpha values to iterate over
lora_alpha_values = [8, 24, 32, 40]

n_teacher = 2000
teacher_df = load_db("/content/teacher_llm_dataset.db", "LLM_results", n_teacher, seed=42)
tokenized_data = load_process_tokenize(n_teacher, teacher_df, teacher=True)

for lora_alpha in lora_alpha_values:
    # Quantization config
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )

    # Load the base model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map={"": 0},
        trust_remote_code=True,
        quantization_config=bnb_config
    )

    # Update the Lora Config to iterate over lora_alpha instead of r
    config = LoraConfig(
        r=16,  # Keep r fixed at 40 as per your initial setup
        lora_alpha=lora_alpha,  # Iterate over lora_alpha values
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    # Get PEFT model with current Lora config
    model = get_peft_model(model, config)

    # Optional: Print trainable parameters or any other details
    model.print_trainable_parameters()

    # Define training arguments
    finetuned_model_name = f'phi-1_5-finetuned-aqua-rat-AM-teacher-2k-alpha-{lora_alpha}'
    training_arguments = TrainingArguments(
        output_dir=finetuned_model_name,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        num_train_epochs=2,
        push_to_hub=True
    )

    trainer = Trainer(
        model=model,
        train_dataset=tokenized_data,
        args=training_arguments,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
    )
    trainer.train()
    trainer.push_to_hub()

Tokenizing data:   0%|          | 0/2000 [00:00<?, ? examples/s]

trainable params: 3,145,728 || all params: 1,421,416,448 || trainable%: 0.2213093850451912


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
100,1.086900
200,0.815000
300,0.802500
400,0.783000
500,0.779300
600,0.732200
700,0.732300
800,0.749100
900,0.729800
1000,0.733500


trainable params: 3,145,728 || all params: 1,421,416,448 || trainable%: 0.2213093850451912


...

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
100,0.971900
200,0.782700
300,0.772200
400,0.755100
500,0.749100
600,0.689800
700,0.689300
800,0.708400
900,0.687300
1000,0.689500


trainable params: 3,145,728 || all params: 1,421,416,448 || trainable%: 0.2213093850451912


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
100,0.956600
200,0.777800
300,0.768100
400,0.750900
500,0.744900
600,0.682300
700,0.681700
800,0.701400
900,0.679700
1000,0.681600
